In [42]:
import pandas as pd
import numpy as np

In [43]:
###### 1 Min Data - 1 Step Forecasting ######

In [44]:
def CreateData(data_c, shif, train_index):
    data_c.drop("system_time", axis = 1)
    data_c['Next_mid'] = data_c.midpoint.shift(-shif)
    data_c['Price_move'] = (data_c.Next_mid - data_c.midpoint) / data_c.midpoint
    data_c.dropna(axis=0, inplace=True) #drop the last row which is now Nan

    data_c['y'] = 0
    data_c.loc[(data_c['Price_move'] < -0.001), 'y'] = 1
    data_c.loc[(data_c['Price_move'] > 0.001), 'y'] = 2
    data_c.drop(['Next_mid', 'midpoint', 'Price_move', "system_time", "Return"], axis=1, inplace=True)
    Y = data_c.pop("y")
    
    data_std = data_c

    r, x = data_c.shape

    for j in range(x):
        m = data_c.iloc[:train_index,j].mean()
        s = data_c.iloc[:train_index,j].std()
        for i in range(r):
            data_std.iloc[i,j] = (data_c.iloc[i,j] - m) / s
    
    return data_std, Y

In [45]:
def CreateBidAskData(data_c):
    bids = data_c.iloc[:,0:18]
    asks = data_c.iloc[:,78:93]
    data_bid_ask  = pd.concat([bids.reset_index(drop=True), asks], axis=1)
    
    return data_bid_ask
    

In [46]:
def CreateOderFlowData(data_c):
    data_order_flow = data_c.iloc[:,:153]
    
    return(data_order_flow)

In [47]:
data_1min = pd.read_csv("data/final/1min/ADA.csv")
data_5min = pd.read_csv("data/final/5min/ADA.csv")

In [48]:
data_5min

,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,...,RK,PreAvg,JV,PJ,SV,ASV,RQ,RQTri,RQQuad,NV
0,2021-04-07 12:52:59.055697+00:00,1.18950,0.0006,167316.421241,146494.881971,-0.000252,-0.000504,-0.000588,-0.000757,-0.000841,...,0.000105,-0.000019,0.0,0.0,5.140600e-06,0.000029,3.154452e-06,4.580964e-07,3.249198e-07,0.000021
1,2021-04-07 12:57:59.055697+00:00,1.19220,0.0002,225174.698211,154055.431994,-0.000084,-0.000168,-0.000252,-0.000336,-0.000419,...,0.000352,-0.000014,0.0,0.0,1.098163e-06,0.000021,1.753774e-06,2.541538e-07,2.885373e-07,0.000020
2,2021-04-07 13:02:59.055697+00:00,1.19345,0.0013,207601.867885,172724.620118,-0.000545,-0.000628,-0.000712,-0.000880,-0.001131,...,0.000225,-0.000010,0.0,0.0,8.575423e-06,0.000016,1.128024e-06,1.633196e-07,2.579751e-07,0.000015
3,2021-04-07 13:07:59.055697+00:00,1.19695,0.0011,276967.880405,261804.113318,-0.000460,-0.000543,-0.000710,-0.000794,-0.000877,...,0.000254,-0.000009,0.0,0.0,9.784793e-06,0.000014,9.890948e-07,1.421479e-07,2.214980e-07,0.000011
4,2021-04-07 13:12:59.055697+00:00,1.20070,0.0004,126267.772824,99449.433003,-0.000167,-0.000583,-0.000666,-0.000916,-0.000999,...,0.000500,-0.000007,0.0,0.0,6.258348e-06,0.000010,5.863093e-07,8.164674e-08,1.226600e-07,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346,2021-04-19 09:20:00.442103+00:00,1.26760,0.0008,33206.342863,52792.581120,-0.000316,-0.000394,-0.000473,-0.000552,-0.000789,...,0.000255,-0.000015,0.0,0.0,3.577968e-06,0.000023,2.173087e-06,3.142804e-07,1.300461e-07,0.000019
3347,2021-04-19 09:25:00.442103+00:00,1.27000,0.0008,65347.061121,91306.476898,-0.000315,-0.000394,-0.000709,-0.000787,-0.000866,...,0.000107,-0.000014,0.0,0.0,6.199524e-09,0.000021,2.054053e-06,2.967316e-07,1.281094e-07,0.000019
3348,2021-04-19 09:30:00.442103+00:00,1.27010,0.0006,57570.223597,43808.038077,-0.000236,-0.000315,-0.000472,-0.000787,-0.000945,...,0.000083,-0.000012,0.0,0.0,5.752836e-06,0.000019,1.919410e-06,2.786151e-07,1.174656e-07,0.000017
3349,2021-04-19 09:35:00.442103+00:00,1.27315,0.0007,101466.684214,70765.245668,-0.000275,-0.000353,-0.000511,-0.000589,-0.000668,...,0.000216,-0.000007,0.0,0.0,3.696051e-06,0.000011,4.285714e-07,6.157737e-08,1.012418e-07,0.000015


In [49]:
data_5min.shape

(3351, 170)

In [50]:
3351/12

279.25

In [51]:
3350-280*2

2790

In [52]:
data_1min_std, Y_1min = CreateData(data_1min, 1, 14000)
data_1min = pd.read_csv("data/final/1min/ADA.csv")
data_5min = pd.read_csv("data/final/5min/ADA.csv")

data_1min = pd.read_csv("data/final/1min/ADA.csv")

data_1min_5_std, Y_1_5min = CreateData(data_1min, 5, 14000)

In [53]:
Y_1min.value_counts()

0    9675
2    3738
1    3631
Name: y, dtype: int64

In [54]:
def CreateData(data_c, shif, train_index):
    data_c.drop("system_time", axis = 1)
    data_c['Next_mid'] = data_c.midpoint.shift(-shif)
    data_c['Price_move'] = (data_c.Next_mid - data_c.midpoint) / data_c.midpoint
    data_c.dropna(axis=0, inplace=True) #drop the last row which is now Nan

    data_c['y'] = 0
    data_c.loc[(data_c['Price_move'] < -0.001), 'y'] = 1
    data_c.loc[(data_c['Price_move'] > 0.001), 'y'] = 2
    data_c.drop(['Next_mid', 'midpoint', 'Price_move', "system_time", "Return"], axis=1, inplace=True)
    Y = data_c.pop("y")
    
    data_std = data_c

    r, x = data_c.shape

    for j in range(x):
        m = data_c.iloc[:train_index,j].mean()
        s = data_c.iloc[:train_index,j].std()
        for i in range(r):
            data_std.iloc[i,j] = (data_c.iloc[i,j] - m) / s
    
    return data_std, Y

In [55]:
data_5min = pd.read_csv("data/final/5min/ADA.csv")
data_5min_std, Y_5min = CreateData(data_5min, 1, 2790)

In [56]:
Y_5min.value_counts()

2    1218
1    1190
0     942
Name: y, dtype: int64

In [57]:
data_1min_std.to_csv("data/final/1min/HF/ADA_std.csv", index = False)
data_1min_5_std.to_csv("data/final/1min/HF/ADA_std_5.csv", index = False)

In [58]:
data_5min_std.to_csv("data/final/5min/HF/ADA_std.csv", index = False)

In [59]:
Y_1min.to_csv("data/final/1min/ADA_Y_std.csv", index = False)
Y_1_5min.to_csv("data/final/1min/ADA_Y_5_std.csv", index = False)


In [60]:
Y_5min.to_csv("data/final/5min/ADA_Y_std.csv", index = False)

In [61]:
OrderFlow1 = CreateOderFlowData(data_1min_std)
OrderFlow1_5 = CreateOderFlowData(data_1min_5_std)



OrderFlow1.to_csv("data/final/1min/OrderFlow/ADA_std.csv", index = False)
OrderFlow1_5.to_csv("data/final/1min/OrderFlow/ADA_std_5.csv", index = False)


In [62]:
OrderFlow5 = CreateOderFlowData(data_5min_std)
OrderFlow5.to_csv("data/final/5min/OrderFlow/ADA_std.csv", index = False)

In [63]:
BidAsk1 = CreateBidAskData(data_1min_std)
BidAsk1_5 = CreateBidAskData(data_1min_5_std)

BidAsk1.to_csv("data/final/1min/BidAsk/ADA_std.csv", index = False)
BidAsk1_5.to_csv("data/final/1min/BidAsk/ADA_std_5.csv", index = False)


In [64]:
BidAsk5 = CreateBidAskData(data_5min_std)
BidAsk5.to_csv("data/final/5min/BidAsk/ADA_std.csv", index = False)